In [1]:
from pathlib import Path
from pprint import pprint
from ost import Sentinel1Batch
from ost.helpers import vector, raster
import json
import gdal
import os

# load username and pw to access sentinel files
username = json.load(open('login_data.json'))['username']
pw = json.load(open('login_data.json'))['pw']

# define the project directory
project_dir = Path.home() / "Documents" / "Python" / "sen1_ost" / "data"

In [2]:
niedersachsen_fields = json.load(open('data/aoi/niedersachsen.geojson'))

In [3]:
target_len = 15

for field_no in range(0,10):
    single_field = niedersachsen_fields['features'][field_no]
    output = {"type":"FeatureCollection","features":[]}

    len_list = sorted([[idx, len(elem[0])] for idx, elem in enumerate(single_field['geometry']['coordinates'])],key=lambda x: x[1],reverse=True)[:target_len]
    reg_len = [i[0] for i in len_list]

    for idx, poly in enumerate(single_field['geometry']['coordinates']):
        if len(single_field['geometry']['coordinates'])<target_len or idx in reg_len:
            xfeature = {"type":"Feature", "geometry":{"type":"Polygon"}}
            xfeature['geometry']['coordinates'] = poly
            output['features'].append(xfeature)
            open('data/temp/test_' + str(field_no) + '.geojson', 'w').write(json.dumps(output,separators=(',',':'),ensure_ascii=False).replace('}},','}},\n'))

    # add rest of preprocessing
